# 1. Load data. 

Import data from the CSV file. Then a Graph class will be used to store and model the data with nodes and edges for future use. 

In [349]:
import csv
import networkx as nx


tubeData =nx.Graph()

#This function will load the data of the csv file and create a graph for modelling the data set
def load():
    with open("tubedata.csv") as f:
        
        reader = csv.reader(f, skipinitialspace=True)
        
        #Data will be stored in the tubeData graph with 4 attributes: weight, tubeline, mainZone and secondaryZone. 
        for row in reader: 
            
            tubeData.add_edge(row[0], row[1], weight=float(row[3]), tubeLine=row[2], mainZone=row[4], secondaryZone=row[5])
 

In [350]:
load()

# 2.Implementation of DFS, BFS and UCS. 

Class Node will be used for representing the nodes for three agenda-based search mechanisims. 
<br>This calls will include the common features needed for the 3 algorithms:
 - State: represents the current station
 - Parent the node of the previous station
 - Time: the average time it takes to commute from one station to the other,
 - Tubeline: the line that connects both stations.


In [351]:
class Node(): 
    def __init__(self, state, parent, time, tubeLine):        
        self.state=state
        self.parent=parent
        self.time=time
        self.tubeLine = tubeLine

## 2.1 DFS

The following class defines how the nodes must be expanded in the DFS algorithm. 
<br> Since DFS follows a Stack data structure, nodes will be added at the end of the frontier and they will be the first ones to explore. 

In [352]:
class DFS(): 
    #Initialization function. 
    def __init__(self):
        self.frontier =[]
        
    #Add node to the frontier. 
    def addNode(self, node):
        self.frontier.append(node)
        
    #Check if the frontier is empty.     
    def isEmpty(self):
        return len(self.frontier)==0
    
    #If the frontier is not empty, a node at the end of the frontier is removed. 
    def removeNode(self): 
        if self.isEmpty(): 
            raise Exception ("The frontier is empty")
        else: 
            node=self.frontier[-1]
            self.frontier=self.frontier[:-1]
            return node
   

## 2.2 BFS

The following class defines how the nodes must be expanded in the BFS algorithm. 
<br> Since BFS follows a Queue data structure, nodes will be added at the end of the frontier, while the nodes at the start of the frontier will be the first ones to be explored. 

In [353]:
class BFS(DFS):
    #If the frontier is not empty, a node at the start of the frontier is removed
    def removeNode(self): 
        if self.isEmpty(): 
            raise Exception ("The frontier is empty")
        else: 
                        
            node=self.frontier[0]
            self.frontier=self.frontier[1:]
            return node
        
        

## 2.3 UCS

The following class defines how the nodes must be expanded in the UCS algorithm.
<br>UCS is similar to  BFS. However, the nodes that will be explored, and therefore removed from the frontier, will depend on the cost function. 
<br> The cost function is defined as the sum of the average time taken from the initial station to the current node. 
<br> The nodes will lower cost function will be expored first. Consequently, after adding a node in the frontier, the array will be sorted according to the cost of each node. 

In [354]:
class UCS(BFS):

    def __init__(self):
        self.frontier=[]
        #Stores as a tuple the nodes that have been added to the frontier and its corresponding commutation time. 
        self.uscFrontier= []
        
    def addNode(self,node): 
        #If it is the starting node, it will be added to the frontier and the usc will set the commutation time to 0.        
        if node.time == None: 
            self.frontier.append(node)
            self.uscFrontier.append([node, 0])
        else:
            self.frontier.append(node)
            
            weight =0 
            child = node 
            
            #Calculate the time taken from the initial node to the current node. 
            while child.time is not None: 
                weight = child.time + weight
                child = child.parent
            
            
            
            self.uscFrontier.append([node, weight])
            
            #Sort the nodes in the frontier according to the commutation time that has been saved in the uscFrontier array. 
            newFrontier= sorted(self.uscFrontier, key=lambda x:x[1], reverse=False)
            
            newFrontier2 = []
            for x in newFrontier:
                if x[0] in self.frontier:
                    newFrontier2.append(x[0])
                    
            self.frontier=[]
            self.frontier=newFrontier2
           
            

## 2.4 Finding the path function. 

The following function backtracks from the current node (goal) to the start station. It will return the path that must be followed and the time that it takes to go from start to the destination. 

In [355]:
def routeToDestination(currentNode):
    route = []
    timeTaken=0
    node=currentNode
    #When the node does not have a parent means its the starting station
    while node.parent is not None: 
        route.append(node.state)
        timeTaken += (node.time)
        node = node.parent

    route.append(node.state)
    route.reverse()
    return route, timeTaken

The following function will find the path that each algorithm will return. 

In [356]:
def findPath(nxData, start, end, algorithm ):
    startNode = Node(state=start, parent=None, time=None, tubeLine=None)
    
    #The frontier will be initialised depending on the algorithm we want to implement.
    if algorithm =="DFS":
        frontier=DFS()
    elif algorithm == "BFS":
        frontier = BFS()
    elif algorithm == "UCS":
        frontier = UCS()

    #the node will be added to the frontier and to the explored set.     
    frontier.addNode(startNode)
    
    explored=set()
    explored.add(start)
    num_explored =1     
    
    #while a path has not been found and the frontier is not empty, the function will look for the best route. 
    while True: 
        if frontier.isEmpty(): 
            return None, None
        else:
            currentNode= frontier.removeNode()
            num_explored +=1
            
            #If destinations has been reached, return the path and the time it takes to reach. 
            if currentNode.state == end:
                route, timeTaken = routeToDestination(currentNode)
                return route, timeTaken, num_explored
            
            #if the removed node does not correspond to the goal station, we will look for the next stations we can go through. 
            neighbours =nxData.neighbors(currentNode.state)
            #For each estation, a child node is created and added to the frontier for further exploration, 
            for neighbour in neighbours:
                stationTime =nxData[currentNode.state][neighbour]["weight"]
                tube =nxData[currentNode.state][neighbour]["tubeLine"]
                
                
                child = Node(state=neighbour, parent=currentNode, time=stationTime, tubeLine = tube)

                if neighbour not in explored:                                      
                    frontier.addNode(child)
                    explored.add(child.state)
                            
    


## 2.5 Experimental results.

Experimental results for route from Euston to Victoria

In [357]:
#DFS
path, timing, num_explored = findPath(tubeData, "Euston", "Victoria", "DFS")
print("DFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#BFS
path, timing ,num_explored= findPath(tubeData, "Euston", "Victoria", "BFS")
print("BFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#UCS
path, timing , num_explored= findPath(tubeData, "Euston", "Victoria", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

DFS algorithm outputs the following path:
['Euston', "King's Cross St. Pancras", 'Russell Square', 'Holborn', 'Covent Garden', 'Leicester Square', 'Piccadilly Circus', 'Green Park', 'Victoria']
The time taken to reach destination is: 13.0. And 26 nodes have been exploded 

BFS algorithm outputs the following path:
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
The time taken to reach destination is: 7.0. And 36 nodes have been exploded 

UCS algorithm outputs the following path:
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
The time taken to reach destination is: 7.0. And 31 nodes have been exploded 



Experimental results for route from Canada Water to Stratford

In [358]:
#DFS
path, timing, num_explored = findPath(tubeData, "Canada Water", "Stratford", "DFS")
print("DFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#BFS
path, timing ,num_explored= findPath(tubeData, "Canada Water", "Stratford", "BFS")
print("BFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#UCS
path, timing , num_explored= findPath(tubeData, "Canada Water", "Stratford", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

DFS algorithm outputs the following path:
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
The time taken to reach destination is: 15.0. And 7 nodes have been exploded 

BFS algorithm outputs the following path:
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
The time taken to reach destination is: 15.0. And 41 nodes have been exploded 

UCS algorithm outputs the following path:
['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford']
The time taken to reach destination is: 14.0. And 53 nodes have been exploded 



Experimental results for route from New Cross Gate to Stepney Green

In [359]:
#DFS
path, timing, num_explored = findPath(tubeData, "New Cross Gate", "Stepney Green", "DFS")
print("DFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#BFS
path, timing ,num_explored= findPath(tubeData, "New Cross Gate", "Stepney Green", "BFS")
print("BFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#UCS
path, timing , num_explored= findPath(tubeData, "New Cross Gate", "Stepney Green", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


DFS algorithm outputs the following path:
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford', 'Mile End', 'Stepney Green']
The time taken to reach destination is: 27.0. And 33 nodes have been exploded 

BFS algorithm outputs the following path:
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
The time taken to reach destination is: 14.0. And 27 nodes have been exploded 

UCS algorithm outputs the following path:
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
The time taken to reach destination is: 14.0. And 19 nodes have been exploded 



Experimental results for route from Ealing Broadway to South Kensington

In [360]:
#DFS
path, timing, num_explored = findPath(tubeData, "Ealing Broadway", "South Kensington", "DFS")
print("DFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#BFS
path, timing ,num_explored= findPath(tubeData, "Ealing Broadway", "South Kensington", "BFS")
print("BFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#UCS
path, timing , num_explored= findPath(tubeData, "Ealing Broadway", "South Kensington", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

DFS algorithm outputs the following path:
['Ealing Broadway', 'Ealing Common', 'North Ealing', 'Park Royal', 'Alperton', 'Sudbury Town', 'Sudbury Hill', 'South Harrow', 'Rayners Lane', 'West Harrow', 'Harrow-on-the-Hill', 'Northwick Park', 'Preston Road', 'Wembley Park', 'Finchley Road', 'Baker Street', 'Bond Street', 'Green Park', 'Victoria', 'Sloane Square', 'South Kensington']
The time taken to reach destination is: 57.0. And 180 nodes have been exploded 

BFS algorithm outputs the following path:
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
The time taken to reach destination is: 20.0. And 51 nodes have been exploded 

UCS algorithm outputs the following path:
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
The time taken to reach destination is: 20.0. And 54 nodes have 

Experimental results for route from Baker Street to Wembley Park

In [361]:
#DFS
path, timing, num_explored = findPath(tubeData, "Baker Street", "Wembley Park", "DFS")
print("DFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#BFS
path, timing ,num_explored= findPath(tubeData, "Baker Street", "Wembley Park", "BFS")
print("BFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#UCS
path, timing , num_explored= findPath(tubeData, "Baker Street", "Wembley Park", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

DFS algorithm outputs the following path:
['Baker Street', 'Finchley Road', 'Wembley Park']
The time taken to reach destination is: 13.0. And 4 nodes have been exploded 

BFS algorithm outputs the following path:
['Baker Street', 'Finchley Road', 'Wembley Park']
The time taken to reach destination is: 13.0. And 17 nodes have been exploded 

UCS algorithm outputs the following path:
['Baker Street', 'Finchley Road', 'Wembley Park']
The time taken to reach destination is: 13.0. And 71 nodes have been exploded 



Experimental results for route from Epping to Wanstead

In [362]:
#DFS
path, timing, num_explored = findPath(tubeData, "Epping", "Wanstead", "DFS")
print("DFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#BFS
path, timing ,num_explored= findPath(tubeData, "Epping", "Wanstead", "BFS")
print("BFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#UCS
path, timing , num_explored= findPath(tubeData, "Epping", "Wanstead", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

DFS algorithm outputs the following path:
['Epping', 'Theydon Bois', 'Debden', 'Loughton', 'Buckhurst Hill', 'Woodford', 'South Woodford', 'Snaresbrook', 'Leytonstone', 'Wanstead']
The time taken to reach destination is: 20.0. And 11 nodes have been exploded 

BFS algorithm outputs the following path:
['Epping', 'Theydon Bois', 'Debden', 'Loughton', 'Buckhurst Hill', 'Woodford', 'South Woodford', 'Snaresbrook', 'Leytonstone', 'Wanstead']
The time taken to reach destination is: 20.0. And 16 nodes have been exploded 

UCS algorithm outputs the following path:
['Epping', 'Theydon Bois', 'Debden', 'Loughton', 'Buckhurst Hill', 'Woodford', 'South Woodford', 'Snaresbrook', 'Leytonstone', 'Wanstead']
The time taken to reach destination is: 20.0. And 14 nodes have been exploded 



Experimental results for route from Epping to Wanstead

In [363]:
#DFS
path, timing, num_explored = findPath(tubeData, "Westminster", "Piccadilly Circus", "DFS")
print("DFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#BFS
path, timing ,num_explored= findPath(tubeData, "Westminster", "Piccadilly Circus", "BFS")
print("BFS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

#UCS
path, timing , num_explored= findPath(tubeData, "Westminster", "Piccadilly Circus", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

DFS algorithm outputs the following path:
['Westminster', 'Waterloo', 'Bank/Monument', 'Moorgate', 'Old Street', 'Angel', "King's Cross St. Pancras", 'Russell Square', 'Holborn', 'Covent Garden', 'Leicester Square', 'Piccadilly Circus']
The time taken to reach destination is: 23.0. And 85 nodes have been exploded 

BFS algorithm outputs the following path:
['Westminster', 'Green Park', 'Piccadilly Circus']
The time taken to reach destination is: 4.0. And 11 nodes have been exploded 

UCS algorithm outputs the following path:
['Westminster', 'Green Park', 'Piccadilly Circus']
The time taken to reach destination is: 4.0. And 12 nodes have been exploded 



# 3. Extending Cost function.

The new cos function will be similar to the one implemented in UCS. However, it will add a penalisation if the tube line must be changed. The weight of this penalisation will depend on the size of the station. 
The bigger the station, higher will be the time taken to walk from one platform to another. 
<br> The act of changing from one platform to another is considered to take 2 minutes. An additional minute will be added as the number of platforms increases. However, if the station has more than 5 platforms, 30 seconds 
will be added per platform 

In [381]:
class UCS_im(BFS):

    def __init__(self):
        self.frontier=[]
        #Stores as a tuple the nodes that have been added to the frontier and its corresponding commutation time including the penalisation for changing tube line.
        self.uscFrontier= []
        
    def addNode(self,node, platforms): 
        #If it is the starting node, it will be added to the frontier and the usc will set the commutation time to 0.          
        if node.time == None: 
            self.frontier.append(node)
            self.uscFrontier.append([node, 0])
        else:
            self.frontier.append(node)
            
            weight =0 
            child = node 
            
            #Calculate the time taken from the initial node to the current node. 
            while child.time is not None: 
                weight = child.time + weight
                child = child.parent
            
            #Adds platform change penalisation. 
            tubeLine_parent=node.parent.tubeLine
            tubeLine_child = node.tubeLine
            if tubeLine_parent is not None:
                #If we change the tube Line. Add 2 minites. 
                if tubeLine_parent != tubeLine_child:
                    weight +=2 
                    #If there are 4 or 5 platforms add additional minutes.
                    if platforms==4:
                        weight +=1
                    elif platforms==5: 
                        weight +=2                        
                    #if there are more than 5 platforms add 30 seconds per platform. 
                    elif platforms>5:
                        weight += (2 + (platforms-5)*0.5)
            
            
            self.uscFrontier.append([node, weight])
            #Sort the nodes in the frontier according to the commutation time that has been saved in the uscFrontier array. 
            newFrontier= sorted(self.uscFrontier, key=lambda x:x[1], reverse=False)
            
            newFrontier2 = []
            for x in newFrontier:
                if x[0] in self.frontier:
                    newFrontier2.append(x[0])
                    
            self.frontier=[]
            self.frontier=newFrontier2 
            

The following function is the same as the findPath function, the only thing that changes is that the addNode function will include the number of platforms as parameters. 

In [382]:
def ucs_i(nxData, start, end):
    startNode = Node(state=start, parent=None, time=None, tubeLine=None)
    
    #Initialise the improved UCS class. 
    frontier=UCS_im()
   
    #the node and its tubeLine will be added to the frontier and to the explored set.
    frontier.addNode(startNode, None)
    
    explored=set()
    explored.add((start, None))
    num_explored =1 
    
    #while a path has not been found and the frontier is not empty, the function will look for the best route.  
    while True: 
        if frontier.isEmpty(): 
            return None, None
        else:
            currentNode= frontier.removeNode()

            num_explored +=1
            
            #If destinations has been reached, return the path and the time it takes to reach.
            if currentNode.state == end:
                route, timeTaken = routeToDestination(currentNode)
                return route, timeTaken, num_explored
            
            #if the removed node does not correspond to the goal station, we will look for the next stations we can go through.
            neighbours =nxData.neighbors(currentNode.state)
            #Save the number of platforms of each station. 
            platforms = len(list(nxData.neighbors(currentNode.state)))
            
            for neighbour in neighbours:
                stationTime =nxData[currentNode.state][neighbour]["weight"]
                tube =nxData[currentNode.state][neighbour]["tubeLine"]
               
                #For each estation, a child node is created and added to the frontier for further exploration, 
                child = Node(state=neighbour, parent=currentNode, time=stationTime, tubeLine = tube)

                if (neighbour, tube) not in explored:                                      
                    frontier.addNode(child, platforms)
                    explored.add((child.state, child.tubeLine))
                            
 

## 3.1 Experimental results. 

Results obtained for the path between Euston and Victoria station. 

In [383]:
path, timing , num_explored= findPath(tubeData,"Euston", "Victoria", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Euston", "Victoria")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
The time taken to reach destination is: 7.0. And 31 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
The time taken to reach destination is: 7.0. And 18 nodes have been exploded 



Results obtained for the path between Canada Water and Stratford station. 

In [384]:
path, timing , num_explored= findPath(tubeData,"Canada Water", "Stratford", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Canada Water", "Stratford")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford']
The time taken to reach destination is: 14.0. And 53 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
The time taken to reach destination is: 15.0. And 63 nodes have been exploded 



In [385]:
path, timing , num_explored= findPath(tubeData,"New Cross Gate", "Stepney Green", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "New Cross Gate", "Stepney Green")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
The time taken to reach destination is: 14.0. And 19 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
The time taken to reach destination is: 14.0. And 28 nodes have been exploded 



Results obtained for the path between Ealing Broadway and South Kensignton station. 

In [386]:
path, timing , num_explored= findPath(tubeData,"Ealing Broadway", "South Kensington", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Ealing Broadway", "South Kensington")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
The time taken to reach destination is: 20.0. And 54 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
The time taken to reach destination is: 20.0. And 59 nodes have been exploded 



Results obtained for the path between Baker Street and Wembley Park station. 

In [387]:
path, timing , num_explored= findPath(tubeData,"Baker Street", "Wembley Park", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Baker Street", "Wembley Park")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Baker Street', 'Finchley Road', 'Wembley Park']
The time taken to reach destination is: 13.0. And 71 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Baker Street', 'Finchley Road', 'Wembley Park']
The time taken to reach destination is: 13.0. And 80 nodes have been exploded 



Results obtained for the path between Epping and Victoria Wanstead. 

In [388]:
path, timing , num_explored= findPath(tubeData,"Epping", "Wanstead", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Epping", "Wanstead")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Epping', 'Theydon Bois', 'Debden', 'Loughton', 'Buckhurst Hill', 'Woodford', 'South Woodford', 'Snaresbrook', 'Leytonstone', 'Wanstead']
The time taken to reach destination is: 20.0. And 14 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Epping', 'Theydon Bois', 'Debden', 'Loughton', 'Buckhurst Hill', 'Woodford', 'South Woodford', 'Snaresbrook', 'Leytonstone', 'Wanstead']
The time taken to reach destination is: 20.0. And 15 nodes have been exploded 



Results obtained for the path between Westminster and Piccadilly Circus station. 

In [389]:
path, timing , num_explored= findPath(tubeData,"Westminster", "Piccadilly Circus", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Westminster", "Piccadilly Circus")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Westminster', 'Green Park', 'Piccadilly Circus']
The time taken to reach destination is: 4.0. And 12 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Westminster', 'Embankment', 'Charing Cross', 'Piccadilly Circus']
The time taken to reach destination is: 5.0. And 21 nodes have been exploded 



# 4. Heurisitc implementation

In [390]:
class NodeX(): 
    def __init__(self, state, parent, time, tubeLine, mainZone, secZone):
        self.state=state
        self.parent=parent
        self.time=time
        self.tubeLine = tubeLine
        self.mainZone = mainZone
        self.secZone=secZone

In [391]:
#This function will provide the stations main and secondary zones. 
def stationZones(tubeData, target):
    endNeighobur=tubeData.neighbors(target)
    endFron=set()
    endFron2=[]
    n=""
    yn=False
    for neighbour in endNeighobur:
        n=neighbour        
        if tubeData[target][neighbour]["secondaryZone"]!="0":
            yn=True
            break
            
    #If the neighbours' secondary zone is same as the main zone, the secondary zone will be returned with the same value as the main zone. 
    if yn:
        mainZone=tubeData[target][n]["mainZone"]
        secondaryZone = tubeData[target][n]["secondaryZone"]
    else:
        mainZone=tubeData[target][n]["mainZone"]
        secondaryZone = tubeData[target][n]["mainZone"]
    
    return mainZone, secondaryZone



In [392]:
class UCS_heuristic(BFS):

    def __init__(self):
        self.frontier=[]
        #Stores as a tuple the nodes that have been added to the frontier and its corresponding commutation time including the penalisation for changing tube line.
        self.uscFrontier= []
        
    def addNode(self,node, platforms, end_mainZone, end_secZone, mainZone, secondaryZone): 
        #If it is the starting node, it will be added to the frontier and the usc will set the commutation time to 0.                         
        if node.time == None: 
            self.frontier.append(node)
            self.uscFrontier.append([node, 0])
        else:
            self.frontier.append(node)
            
            weight =0 
            child = node 
            
            #Calculate the time taken from the initial node to the current node. 
            while child.time is not None: 
                weight = child.time + weight
                child = child.parent
                
            #Adds platform change penalisation.
            tubeLine_parent=node.parent.tubeLine
            tubeLine_child = node.tubeLine
            if tubeLine_parent is not None: 
                #If we change the tube Line. Add 2 minutes. 
                if tubeLine_parent != tubeLine_child:
                    weight +=2 
                    #If there are 4 or 5 platforms add additional minutes.
                    if platforms==4:
                        weight +=1
                    elif platforms==5: 
                        weight +=2                        
                    #if there are more than 5 platforms add 30 seconds per platform. 
                    elif platforms>5:
                        weight += (2 + (platforms-5)*0.5)
            
                    
                    
            
            parents=node.parent
            p_mainZone = parents.mainZone
            p_secZone = parents.secZone
            
            #dictionary that will convert the zone strings to an integer number, in order to facilitate further mathematical operations. 
            zonesDict ={"1": 1, "2": 2, "3": 3, "4":4, "5": 5, "6": 6, "a": 7, "b": 8, "c": 9, "d": 10}  
            end_mainZone = zonesDict[end_mainZone]
            end_secZone = zonesDict[end_secZone]
            mainZone = zonesDict[mainZone]
            secondaryZone = zonesDict[secondaryZone]
            p_mainZone = zonesDict[p_mainZone]
            p_secZone = zonesDict[p_secZone]
            
            #If the current station is in the same zone, no penalisation
            if mainZone==end_mainZone or secondaryZone==end_mainZone or mainZone==end_secZone or secondaryZone==end_secZone:
                    pass
            else:
                #if the previous station and the current station are in the same zone --> penalisation = 0.5
                if p_mainZone == mainZone or p_mainZone == secondaryZone or p_secZone == mainZone or p_secZone == secondaryZone:
                    weight +=1.5
                    
                #if the previous station and the current station are in not in the same zone
                #but we are moving in the correct direction (outside to inside)--> penalisation = 1, otherwise 2
                elif mainZone >end_mainZone or secondaryZone>end_mainZone or mainZone >end_secZone or secondaryZone>end_secZone: 
                    if p_mainZone == mainZone+1 or p_mainZone == secondaryZone+1 or p_secZone == mainZone+1 or p_secZone == secondaryZone+1:
                        weight +=1
                    else:
                        weight +=2
                
                #if the previous station and the current station are in not in the same zone
                #but we are moving in the correct direction (inside to outside)--> penalisation = 1, otherwise 2
                else: 
                    if p_mainZone == mainZone-1 or p_mainZone == secondaryZone-1 or p_secZone == mainZone-1 or p_secZone == secondaryZone-1:
                        weight +=1
                    else:
                        weight +=2

            

            self.uscFrontier.append([node, weight])
            
            #Sort the nodes in the frontier according to the commutation time that has been saved in the uscFrontier array. 
            newFrontier= sorted(self.uscFrontier, key=lambda x:x[1], reverse=False)
            
            newFrontier2 = []
            for x in newFrontier:
                if x[0] in self.frontier:
                    newFrontier2.append(x[0])
                    
            self.frontier=[]
            self.frontier=newFrontier2
            

In [404]:
def ucs_h(nxData, start, end):
    
    start_mainZone, start_secZone = stationZones(nxData, start)
    startNode = NodeX(state=start, parent=None, time=None, tubeLine=None, mainZone=start_mainZone, secZone=start_secZone)
    
    #Initialise the improved UCS class. 
    frontier=UCS_heuristic()
    
    #the node and its tubeLine will be added to the frontier and to the explored set. 
    frontier.addNode(startNode, 0, None, None, None, None)
    
    explored=set()
    explored.add((start, None))
    num_explored =1 
    
    end_mainZone, end_secZone = stationZones(nxData, end)
        
    
    #while a path has not been found and the frontier is not empty, the function will look for the best route.      
    while True: 
        if frontier.isEmpty(): 
            return None, None
        else:
            currentNode= frontier.removeNode()  
            num_explored +=1
                        
            #If destinations has been reached, return the path and the time it takes to reach.
            if currentNode.state == end:
                route, timeTaken = routeToDestination(currentNode)
                return route, timeTaken, num_explored
            
            #if the removed node does not correspond to the goal station, we will look for the next stations we can go through.
            neighbours = nxData.neighbors(currentNode.state)
            #Save the platforms of the station. 
            platforms = len(list(nxData.neighbors(currentNode.state)))
            
            for neighbour in neighbours:
                stationTime =nxData[currentNode.state][neighbour]["weight"]
                tube =nxData[currentNode.state][neighbour]["tubeLine"]
                mainZone, secondaryZone =stationZones(nxData, currentNode.state)
        
                child = NodeX(state=neighbour, parent=currentNode, time=stationTime, tubeLine = tube, mainZone=mainZone,secZone= secondaryZone)
                
                #For each estation, a child node is created and added to the frontier for further exploration, 
                if (neighbour, tube) not in explored:                                      
                    frontier.addNode(child, platforms, end_mainZone, end_secZone, mainZone, secondaryZone)
                    explored.add((child.state, child.tubeLine))
                            
 

## 4.1 Experimental resutls. 

Results obtained for the path between Euston and Victoria station. 

In [405]:
path, timing , num_explored= findPath(tubeData,"Euston", "Victoria", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Euston", "Victoria")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_h(tubeData, "Euston", "Victoria")
print("UCS_heuristic algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
The time taken to reach destination is: 7.0. And 31 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
The time taken to reach destination is: 7.0. And 18 nodes have been exploded 

UCS_heuristic algorithm outputs the following path:
['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']
The time taken to reach destination is: 7.0. And 17 nodes have been exploded 



Results obtained for the path between Canada Water and Stratford station. 

In [406]:
path, timing , num_explored= findPath(tubeData,"Canada Water", "Stratford", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Canada Water", "Stratford")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_h(tubeData, "Canada Water", "Stratford")
print("UCS_heuristic algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford']
The time taken to reach destination is: 14.0. And 53 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
The time taken to reach destination is: 15.0. And 63 nodes have been exploded 

UCS_heuristic algorithm outputs the following path:
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
The time taken to reach destination is: 15.0. And 41 nodes have been exploded 



In [407]:
path, timing , num_explored= findPath(tubeData,"New Cross Gate", "Stepney Green", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "New Cross Gate", "Stepney Green")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_h(tubeData, "New Cross Gate", "Stepney Green")
print("UCS_heuristic algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
The time taken to reach destination is: 14.0. And 19 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
The time taken to reach destination is: 14.0. And 28 nodes have been exploded 

UCS_heuristic algorithm outputs the following path:
['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
The time taken to reach destination is: 14.0. And 21 nodes have been exploded 



Results obtained for the path between Ealing Broadway and South Kensignton station. 

In [408]:
path, timing , num_explored= findPath(tubeData,"Ealing Broadway", "South Kensington", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Ealing Broadway", "South Kensington")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_h(tubeData, "Ealing Broadway", "South Kensington")
print("UCS_heuristic algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
The time taken to reach destination is: 20.0. And 54 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
The time taken to reach destination is: 20.0. And 59 nodes have been exploded 

UCS_heuristic algorithm outputs the following path:
['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington']
The time taken to reach destination is: 20.0. And 55 nodes have been exploded 



Results obtained for the path between Baker Street and Wembley Park station. 

In [409]:
path, timing , num_explored= findPath(tubeData,"Baker Street", "Wembley Park", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Baker Street", "Wembley Park")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_h(tubeData, "Baker Street", "Wembley Park")
print("UCS_heuristic algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Baker Street', 'Finchley Road', 'Wembley Park']
The time taken to reach destination is: 13.0. And 71 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Baker Street', 'Finchley Road', 'Wembley Park']
The time taken to reach destination is: 13.0. And 80 nodes have been exploded 

UCS_heuristic algorithm outputs the following path:
['Baker Street', 'Finchley Road', 'Wembley Park']
The time taken to reach destination is: 13.0. And 78 nodes have been exploded 



Results obtained for the path between Epping and Victoria Wanstead. 

In [410]:
path, timing , num_explored= findPath(tubeData,"Epping", "Wanstead", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Epping", "Wanstead")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_h(tubeData, "Epping", "Wanstead")
print("UCS_heuristic algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Epping', 'Theydon Bois', 'Debden', 'Loughton', 'Buckhurst Hill', 'Woodford', 'South Woodford', 'Snaresbrook', 'Leytonstone', 'Wanstead']
The time taken to reach destination is: 20.0. And 14 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Epping', 'Theydon Bois', 'Debden', 'Loughton', 'Buckhurst Hill', 'Woodford', 'South Woodford', 'Snaresbrook', 'Leytonstone', 'Wanstead']
The time taken to reach destination is: 20.0. And 15 nodes have been exploded 

UCS_heuristic algorithm outputs the following path:
['Epping', 'Theydon Bois', 'Debden', 'Loughton', 'Buckhurst Hill', 'Woodford', 'South Woodford', 'Snaresbrook', 'Leytonstone', 'Wanstead']
The time taken to reach destination is: 20.0. And 14 nodes have been exploded 



Results obtained for the path between Westminster and Piccadilly Circus station. 

In [411]:
path, timing , num_explored= findPath(tubeData,"Westminster", "Piccadilly Circus", "UCS")
print("UCS algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_i(tubeData, "Westminster", "Piccadilly Circus")
print("UCS_improved algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')

path, timing, num_explored = ucs_h(tubeData, "Westminster", "Piccadilly Circus")
print("UCS_heuristic algorithm outputs the following path:\n" + str(path) + '\033[1m'"\nThe time taken to reach destination is: "+str(timing)+". And "+str(num_explored) + " nodes have been exploded \n"'\033[0m')


UCS algorithm outputs the following path:
['Westminster', 'Green Park', 'Piccadilly Circus']
The time taken to reach destination is: 4.0. And 12 nodes have been exploded 

UCS_improved algorithm outputs the following path:
['Westminster', 'Embankment', 'Charing Cross', 'Piccadilly Circus']
The time taken to reach destination is: 5.0. And 21 nodes have been exploded 

UCS_heuristic algorithm outputs the following path:
['Westminster', 'Embankment', 'Charing Cross', 'Piccadilly Circus']
The time taken to reach destination is: 5.0. And 21 nodes have been exploded 

